Dont read this nonsense. I messed up here but I need it.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

In [2]:
path = "datasets/"  #Insert path here
database = path + 'database.sqlite'

In [3]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [4]:
team_t = pd.read_sql("""SELECT * FROM Team""", conn)
team_t

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [5]:
match_t = pd.read_sql("""SELECT * FROM Match""", conn)
match_t

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
team_t[team_t['team_long_name'] == 'Arsenal']

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
27,3459,9825,1.0,Arsenal,ARS


In [7]:
def get_match_history(team_api_id):
    match_home = match_t[match_t['home_team_api_id'] == team_api_id]
    match_home.loc[:,'home_away'] = ['home'] * match_home.shape[0]
    match_home

    match_away = match_t[match_t['home_team_api_id'] == team_api_id]
    match_away.loc[:,'home_away'] = ['away'] * match_home.shape[0]
    match_away

    match_history = pd.concat([match_home, match_away])
    return match_history.sort_values('date',ascending = False)

#arsenal = 9825
match_history = get_match_history(9825)
match_history

c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,home_away
4698,4699,1729,1729,2015/2016,38,2016-05-15 00:00:00,1987597,9825,10252,4,...,1.15,9.00,21.0,NaN,NaN,NaN,NaN,NaN,NaN,away
4698,4699,1729,1729,2015/2016,38,2016-05-15 00:00:00,1987597,9825,10252,4,...,1.15,9.00,21.0,NaN,NaN,NaN,NaN,NaN,NaN,home
4678,4679,1729,1729,2015/2016,36,2016-04-30 00:00:00,1989060,9825,9850,1,...,1.25,6.50,13.0,NaN,NaN,NaN,NaN,NaN,NaN,home
4678,4679,1729,1729,2015/2016,36,2016-04-30 00:00:00,1989060,9825,9850,1,...,1.25,6.50,13.0,NaN,NaN,NaN,NaN,NaN,NaN,away
4618,4619,1729,1729,2015/2016,30,2016-04-21 00:00:00,1989000,9825,8659,2,...,1.25,6.25,15.0,NaN,NaN,NaN,NaN,NaN,NaN,away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,2070,1729,1729,2008/2009,6,2008-09-27 00:00:00,489093,9825,8667,1,...,1.20,6.00,21.0,1.22,5.5,13.0,1.20,5.5,15.0,away
1949,1950,1729,1729,2008/2009,3,2008-08-30 00:00:00,489063,9825,10261,3,...,1.45,4.10,8.5,1.45,4.0,8.0,1.44,3.8,8.0,home
1949,1950,1729,1729,2008/2009,3,2008-08-30 00:00:00,489063,9825,10261,3,...,1.45,4.10,8.5,1.45,4.0,8.0,1.44,3.8,8.0,away
1729,1730,1729,1729,2008/2009,1,2008-08-16 00:00:00,489043,9825,8659,1,...,1.25,6.00,13.0,1.22,5.5,13.0,1.22,5.0,13.0,home


In [10]:
def get_n_history(team_api_id,n):
    return get_match_history(team_api_id)[:n]

tmp = get_n_history(9825,10)
tmp

c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\farhan\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,home_away
4698,4699,1729,1729,2015/2016,38,2016-05-15 00:00:00,1987597,9825,10252,4,...,1.15,9.00,21.0,NaN,NaN,NaN,NaN,NaN,NaN,away
4698,4699,1729,1729,2015/2016,38,2016-05-15 00:00:00,1987597,9825,10252,4,...,1.15,9.00,21.0,NaN,NaN,NaN,NaN,NaN,NaN,home
4678,4679,1729,1729,2015/2016,36,2016-04-30 00:00:00,1989060,9825,9850,1,...,1.25,6.50,13.0,NaN,NaN,NaN,NaN,NaN,NaN,home
4678,4679,1729,1729,2015/2016,36,2016-04-30 00:00:00,1989060,9825,9850,1,...,1.25,6.50,13.0,NaN,NaN,NaN,NaN,NaN,NaN,away
4618,4619,1729,1729,2015/2016,30,2016-04-21 00:00:00,1989000,9825,8659,2,...,1.25,6.25,15.0,NaN,NaN,NaN,NaN,NaN,NaN,away
4618,4619,1729,1729,2015/2016,30,2016-04-21 00:00:00,1989000,9825,8659,2,...,1.25,6.25,15.0,NaN,NaN,NaN,NaN,NaN,NaN,home
4658,4659,1729,1729,2015/2016,34,2016-04-17 00:00:00,1989040,9825,9826,1,...,1.33,5.50,10.0,NaN,NaN,NaN,NaN,NaN,NaN,away
4658,4659,1729,1729,2015/2016,34,2016-04-17 00:00:00,1989040,9825,9826,1,...,1.33,5.50,10.0,NaN,NaN,NaN,NaN,NaN,NaN,home
4638,4639,1729,1729,2015/2016,32,2016-04-02 00:00:00,1989020,9825,9817,4,...,1.36,5.40,9.5,NaN,NaN,NaN,NaN,NaN,NaN,away
4638,4639,1729,1729,2015/2016,32,2016-04-02 00:00:00,1989020,9825,9817,4,...,1.36,5.40,9.5,NaN,NaN,NaN,NaN,NaN,NaN,home


In [12]:
list(tmp.columns)

['id',
 'country_id',
 'league_id',
 'season',
 'stage',
 'date',
 'match_api_id',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal',
 'home_player_X1',
 'home_player_X2',
 'home_player_X3',
 'home_player_X4',
 'home_player_X5',
 'home_player_X6',
 'home_player_X7',
 'home_player_X8',
 'home_player_X9',
 'home_player_X10',
 'home_player_X11',
 'away_player_X1',
 'away_player_X2',
 'away_player_X3',
 'away_player_X4',
 'away_player_X5',
 'away_player_X6',
 'away_player_X7',
 'away_player_X8',
 'away_player_X9',
 'away_player_X10',
 'away_player_X11',
 'home_player_Y1',
 'home_player_Y2',
 'home_player_Y3',
 'home_player_Y4',
 'home_player_Y5',
 'home_player_Y6',
 'home_player_Y7',
 'home_player_Y8',
 'home_player_Y9',
 'home_player_Y10',
 'home_player_Y11',
 'away_player_Y1',
 'away_player_Y2',
 'away_player_Y3',
 'away_player_Y4',
 'away_player_Y5',
 'away_player_Y6',
 'away_player_Y7',
 'away_player_Y8',
 'away_player_Y9',
 'away_player_Y10',
 'away_player

In [15]:
match_api_id = 1989060
current_match = match_t[match_t['match_api_id']==match_api_id]
current_match

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
4678,4679,1729,1729,2015/2016,36,2016-04-30 00:00:00,1989060,9825,9850,1,...,NaN,1.25,6.5,13.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
n = 5
# 5 match before for both home away team